# Technical Report
### NYC Homeless Shelter Population Forecasting

---

### Problem Statement
In this project, I forecasted the total number of individuals sheltered by the NYC Department of Homeless Services (DHS). I used DHS Daily Report publicly available from [NYC Open Data](https://opendata.cityofnewyork.us) to build Time Series model. I aimed to help understand homeless communities better by examining different factors that change the number of indivuals in DHS shelter and to help better prepare for those who are in need of a space at a shelter by forecasting the general demand.

### Data Collection
1. DHS Daily Report (Source: https://data.cityofnewyork.us/Social-Services/DHS-Daily-Report/k46n-sa2m)

DHS Daily Report was scraped using [Socrata Open Data API (SODA)](https://dev.socrata.com/foundry/data.cityofnewyork.us/k46n-sa2m). The report includes the number of total individuals in Shelter and the breakdown of the shelter population such as the total number of children in shelter from 2013. This report is updated daily and the update is availble through the API. Although the data did not have any null values, there were some missing dates indicating the report was not made on some days. For this reason, the daily data was resampled into weekly average data for analysis.

### Database (PostgreSQL)
I built PostgreSQL database on Heroku for this project. The database was contructed for the following reasons. First, the number of rows that are returned from Socrata Open Data API is limited. By contructing a database, I can keep the rows when they are no longer available through the API. Second, as I contruct database with properly cleaned data, I do not have to repeat the data cleaning step when the dataset is used in other Jupyter notebooks or different visualization tools such as Tableau or Dash.

### Exploratory Data Analysis (EDA)
The EDA for the project was largely divided into two parts. The first part was to examine the trend in DHS shelter demographic. This part of EDA is used potentially for exogenous features. The second part of EDA was to understand the trend and seasonality in the total individuals in shelter in order to perform time series analysis.

**Part 1. DHS Shelter Demographic**                          
While the ratio of children to adult and the ratio of people with and without family shows clear trend with a slight seasonality, the ratio of women and men exibits a slight trend and a strong seasonality.

<img src="./Production/Assets/age.png" width = 500>

<img src="./Production/Assets/family.png" width = 500>

<img src="./Production/Assets/gender.png" width = 500>

**Part 2. Total Individuals in DHS Shelter**
<img src="./Production/Assets/total.png" width = 500>

In [ ]:

<img src="./Production/Assets/seasonal.png" width = 500>

<img src="./Production/Assets/acf_pacf.png" width = 400>

### Modeling

### Data Visualization

### Next Steps